# Training the "small" model

The code above will load training data and train the so-called "small" model.

Out of a prescribed number (given by the parameter `seq_len`) of last orders of every user, only the order date and the bill amount is taken into the account by this model. The model ignores every other features of orders.

In [ ]:
from pndapetzim.data import load_datasets
from pndapetzim.data import ORDER_FILE_NAME
from pndapetzim.data import LABEL_FILE_NAME

seq_len = 20
returning_weight = 4.0

train, test = load_datasets(
    order_path='../data/' + ORDER_FILE_NAME,
    label_path='../data/' + LABEL_FILE_NAME,
    seq_len=seq_len,
    train_ratio=100,
    returning_weight=returning_weight,
)
train = train.take(128*100)

In [ ]:
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.metrics import AUC
from tensorflow.keras.metrics import Recall
from tensorflow.keras.optimizers import Adam

from pndapetzim.models import build_amount_date_model

loss = BinaryCrossentropy()
optimiser = Adam(learning_rate=0.01)

model = build_amount_date_model(seq_len, 5)
auc = AUC()
recall = Recall()
metrics = [BinaryAccuracy(), auc, recall]
model.compile(loss=loss, optimizer=optimiser, metrics=metrics)
batch_size = 128

In [ ]:
model.fit(train.batch(batch_size).prefetch(10), epochs=1)

In [ ]:
model.evaluate(test.batch(batch_size))